### Урок 4. Парсинг HTML. XPath
**1)** Написать приложение, которое собирает основные новости с сайтов **news.mail.ru**, **lenta.ru**, **yandex.news**
  Для парсинга использовать xpath. Структура данных должна содержать:
  название источника,
  наименование новости,
  ссылку на новость,
  дата публикации

**2)** Сложить все новости в БД

In [1]:
import datetime
import re
from pprint import pprint

import requests
from lxml import html
from pymongo import MongoClient
import pandas as pd

In [2]:
yandex_news = 'https://yandex.ru/news/?from=tabbar'
lenta_news = 'https://lenta.ru/'
mail_news = 'https://news.mail.ru/?from=menu'

In [3]:
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
          ' (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

На каждом из сайтов используется свой формат записи даты публикации сообщений. Функции `lenta_time_date_converter`, `yandex_time_date_converter` и `mail_time_date_converter` приводят значение даты публикации к формату **datetime**.

In [4]:
mounths = ['января', 'февраля', 'марта', 'апреля', 'мая', 'июня', 'июля', 'августа', 'сентября', 'октября', 'ноября',
           'декабря']

In [5]:
def lenta_time_date_converter(date_time_text):
    date_time = None
    try:
        date_time_text = date_time_text.replace(', ', '')
        tmp_array = date_time_text.strip(' ')
        tmp_array = tmp_array.split(' ')
        day_ = int(tmp_array[1])
        mounth_ = mounths.index(tmp_array[2]) + 1
        year_ = int(tmp_array[3])
        tmp_time = tmp_array[0].split(':')
        hour_ = int(tmp_time[0])
        minutes_ = int(tmp_time[1])
        date_time = datetime.datetime(year_, mounth_, day_, hour_, minutes_)
    except Exception as ex:
        print(f'lenta_date_converter: {ex}')
    return date_time

In [6]:
def yandex_time_date_converter(date_time_text, is_yesterday):
    date_time = None
    try:
        tmp_time = date_time_text.split(':')
        hour_ = int(tmp_time[0].strip())
        minutes_ = int(tmp_time[1].strip())
        date_ = datetime.date.today()
        date_time = datetime.datetime(date_.year, date_.month, date_.day, hour_, minutes_)
        if is_yesterday:
            date_time = date_time - datetime.timedelta(1)
    except Exception as ex:
        print(f'yandex_date_converter: {ex}')
    return date_time


In [7]:
def mail_time_date_converter(date_time_text):
    date_time = None
    try:
        indT = date_time_text.index('T')
        ind_plus = date_time_text.index('+')
        date_ = date_time_text[0:indT]
        time_ = date_time_text[indT + 1:ind_plus]
        deltaTme_ = date_time_text[ind_plus + 1:]
        tmp_date = date_.split('-')
        year_ = int(tmp_date[0])
        mounth_ = int(tmp_date[1])
        day_ = int(tmp_date[2])

        tmp_time = time_.split(':')
        hour_ = int(tmp_time[0])
        minutes_ = int(tmp_time[1])
        seconds_ = int(tmp_time[2])

        tmp_time = deltaTme_.split(':')
        d_hour_ = int(tmp_time[0])
        d_minutes_ = int(tmp_time[1])

        date_time = datetime.datetime(year_, mounth_, day_, hour_, minutes_, seconds_)
        delta_time = datetime.timedelta(hours=d_hour_, minutes=d_minutes_)
        date_time = date_time + delta_time

    except Exception as ex:
        print(f'mail_date_converter: ex')
    return date_time

Функции `request_to_yandex`, `request_to_mail_ru`, `request_to_lenta_ru` преназначены для вытягивания данных о главных новосях с сайтов. 

In [8]:
def mail_news_info(href): # Получаем сведенья о дате публикации и источнике материала для mail.ru
    response = requests.get(href, headers=header)
    dom = html.fromstring(response.text)
    items = dom.xpath("//div[@class ='breadcrumbs breadcrumbs_article js-ago-wrapper']/*/span")

    date_time = items[0].xpath(".//span[@class='note__text breadcrumbs__text js-ago']/@datetime")[0]
    source = items[1].xpath(".//a[@class='link color_gray breadcrumbs__link']/@href")[0]
    return source, date_time

In [9]:
def request_to_mail_ru():
    news_list = []
    try:
        response = requests.get(mail_news, headers=header)
        dom = html.fromstring(response.text)
        items = dom.xpath("//ul[@class='list list_type_square list_half js-module']/li")
        for item in items:
            news = {}
            news_href = item.xpath("./a/@href")[0]
            if news_href.find('https://') == -1:
                news_href = 'https://news.mail.ru' + news_href
            news_name = item.xpath("./a/text()")
            news_source, date_time_text = mail_news_info(news_href)
            news['date'] = mail_time_date_converter(date_time_text)
            news['name'] = news_name[0].replace('\xa0', ' ')
            news['href'] = news_href
            news['source'] = news_source
            news_list.append(news)
    except Exception as ex:
        print(f'request_to_mail_ru {ex}')
    return (news_list)

In [10]:
def request_to_lenta_ru():
    news_list = []
    try:
        response = requests.get(lenta_news, headers=header)
        dom = html.fromstring(response.text)

        items = dom.xpath("//section[@class='row b-top7-for-main js-top-seven']/div[@class='span4']/div")
        for item in items[1:-1]:  
            news = {}
            news_href = 'https://lenta.ru' + item.xpath("./a/@href")[0]
            news_name = item.xpath("./a/text()")
            news_date = item.xpath("./a/time/@datetime") 
            news['date'] = lenta_time_date_converter(news_date[0])
            news['name'] = news_name[0].replace('\xa0', ' ')
            news['href'] = news_href
            news['source'] = 'lenta.ru'
            news_list.append(news)
    except Exception as ex:
        print(f'request_to_lenta_ru {ex}')
    return (news_list)

In [11]:
def request_to_yandex():  # Берем новости из разряда "Интересное"
    news_list = []
    try:
        response = requests.get(yandex_news, headers=header)
        dom = html.fromstring(response.text)

        categories = dom.xpath("//div[@class = 'page-content__cell']//a[contains(@class,'link link_theme_normal"
            " rubric-label rubric-label_top_')]/text()")[5:]   

        print(f'Виды категорий повостей на yandex.ru/news:')
        print(categories)
        index = categories.index('Интересное') + 2
        items = dom.xpath(
            f"//div[@class = 'page-content__cell'][{index}]//table[@class='stories-set__items']//td[@class='stories-set__item']")
        for item in items:
            news = {}
            news_name = item.xpath(".//div[@class='story__topic']//h2[@class='story__title']/a/text()")
            news_source = item.xpath(".//div[@class='story__info']/div[@class='story__date']/text()")[0]
            news_href = item.xpath(".//div[@class='story__topic']//h2[@class='story__title']/a/@href")
            is_yesterday = False
            if news_source.find('вчера') != -1:
                news_source = news_source.replace('вчера', '')
                is_yesterday = True
            news_time = re.findall(r'\d{2}:\d{2}', news_source)[0]
            news_source = news_source.replace(news_time, '')

            news['date'] = yandex_time_date_converter(news_time, is_yesterday)
            news['name'] = news_name[0].replace('\xa0', ' ')
            news['href'] = 'https://yandex.ru' + news_href[0]
            news['source'] = news_source
            news_list.append(news)
    except Exception as ex:
        print(f'request_to_yandex {ex}')
    return (news_list)

Основной код программы
------------------------------------------------------------------------
**1.** Волучаем данные о новостях с сайтов.

In [12]:
lenta_news_list = request_to_lenta_ru()
print(f'Колличество новостей на lenta.ru : {len(lenta_news_list)}')

Колличество новостей на lenta.ru : 9


In [13]:
yandex_news_list = request_to_yandex()
print(f'Колличество новостей на yandex.ru/news : {len(yandex_news_list)}')

Виды категорий повостей на yandex.ru/news:
['Ижевск', 'Интересное', 'Политика', 'Общество', 'Экономика', 'В мире', 'Спорт', 'Происшествия', 'Культура', 'Технологии', 'Наука', 'Авто']
Колличество новостей на yandex.ru/news : 5


In [14]:
mail_news_list = request_to_mail_ru()
print(f'Колличество новостей на news/mail.ru : {len(mail_news_list)}')

Колличество новостей на news/mail.ru : 8


Выполняем подключение к БД

In [15]:
client = MongoClient('localhost', 27017)
db = client['yandex_news']
db = client['mail_news']
db = client['lenta_news']

Новости достаточно быстро устаревают. Поэтому, добавлять сведенья о новостях не имеет смысла. 

**2.** Очищаем все таблицы. 

In [16]:
db.yandex_news.delete_many({})
db.mail_news.delete_many({})
db.lenta_news.delete_many({})

**3.** Заполняем таблицы новыми данными.  

In [17]:
db.yandex_news.insert_many(yandex_news_list)
db.mail_news.insert_many(mail_news_list)
db.lenta_news.insert_many(lenta_news_list)

**4.** Проверка

In [18]:
pd.DataFrame(db.lenta_news.find({},{'_id':0}))

,date,name,href,source
0,2020-07-06 00:53:00,Белоруссия захотела «зацепиться зубами» за рос...,https://lenta.ru/news/2020/07/06/prod/,lenta.ru
1,2020-07-06 00:37:00,В Москве отвергли возможность второй волны кор...,https://lenta.ru/news/2020/07/06/msk_scnd_wave/,lenta.ru
2,2020-07-06 00:08:00,Появилось видео из чемпионской раздевалки «Зен...,https://lenta.ru/news/2020/07/06/zenitrazdevalka/,lenta.ru
3,2020-07-06 00:03:00,Девятилетняя россиянка умерла от удара током в...,https://lenta.ru/news/2020/07/05/fontan/,lenta.ru
4,2020-07-06 00:02:00,США «потеряли» стелс-ракету,https://lenta.ru/news/2020/07/06/navy/,lenta.ru
5,2020-07-05 23:40:00,В Москве скончались 22 пациента с коронавирусом,https://lenta.ru/news/2020/07/05/22_corona/,lenta.ru
6,2020-07-05 23:35:00,Зюганов оценил слова Путина о «мине замедленно...,https://lenta.ru/news/2020/07/05/zuganov/,lenta.ru
7,2020-07-05 23:14:00,Раскрыто состояние сбежавшей от насильников 16...,https://lenta.ru/news/2020/07/05/girl_russia/,lenta.ru
8,2020-07-05 22:05:00,Ученые пересмотрели способ распространения кор...,https://lenta.ru/news/2020/07/05/recommend/,lenta.ru


In [19]:
pd.DataFrame(db.yandex_news.find({},{'_id':0}))

,date,name,href,source
0,2020-07-06 00:09:00,Вторая волна коронавируса может совпасть со вс...,https://yandex.ru/news/story/Vtoraya_volna_kor...,Газета.Ru
1,2020-07-05 11:30:00,Актер Виктор Проскурин похоронен на Троекуровс...,https://yandex.ru/news/story/Akter_Viktor_Pros...,Национальная служба новостей в
2,2020-07-06 00:54:00,Указание пола уберут из удостоверений личности...,https://yandex.ru/news/story/Ukazanie_pola_ube...,Газета.Ru
3,2020-07-05 23:57:00,Alfa Romeo оштрафовали на 5000 евро,https://yandex.ru/news/story/Alfa_Romeo_oshtra...,F1news.ru в
4,2020-07-05 21:10:00,Evanescence показали свои настоящие чувства в ...,https://yandex.ru/news/story/Evanescence_pokaz...,ИА InterMedia в


In [20]:
pd.DataFrame(db.mail_news.find({},{'_id':0}))

,date,name,href,source
0,2020-07-05 18:23:38,"ДНК мужчины, обнаруженная в пробе Зайцевой, пр...",https://sportmail.ru/news/biathlon/42453328/,http://www.tass.ru/
1,2020-07-06 02:14:34,Ученый пришел к неожиданным выводам о происхож...,https://news.mail.ru/society/42455282/,http://www.tass.ru/
2,2020-07-05 16:32:58,Экс-премьер ДНР: республики Донбасса скоро ста...,https://news.mail.ru/politics/42450563/,http://www.kommersant.ru
3,2020-07-05 17:30:52,Актер Вилле Хаапасало стал продавать хачапури ...,https://news.mail.ru/society/42452968/,https://news.mail.ru
4,2020-07-05 14:59:20,Четыре человека погибли при обрушении перекрыт...,https://news.mail.ru/incident/42451656/,http://www.tass.ru/
5,2020-07-06 03:17:03,День в истории: 6 июля,https://news.mail.ru/society/42422886/,https://news.mail.ru
6,2020-07-05 21:11:10,В Чикаго в День независимости расстреляли прох...,https://news.mail.ru/incident/42454320/,https://life.ru/
7,2020-07-05 18:20:12,Россиянам рассказали о синдроме «постковидного...,https://news.mail.ru/society/42449551/,http://www.kp.ru
